In [15]:
%logstop
%logstart -rtq ~/.logs/pw.py append
import seaborn as sns
sns.set()

In [5]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [ ]:
scripts[:2]

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [ ]:
practices[:2]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [8]:
import statistics as stat
import math
def describe(data, key):
    
    values=[]
    for d in data:
        values.append(d[key])

    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    
    for v in values:
        total=total+v
    
    avg=total/len(values)
    
    stotal=0
    for v in values:
        stotal=stotal+(v-avg)**2
    s=math.sqrt(stotal/len(data))
    
    values.sort()
    
   
    med=stat.median(values)
    half_list=len(values)//2
    q75=stat.median(values[-half_list:])
    q25=stat.median(values[:half_list])
    
    

    return (total, avg, s, q25, med, q75)

In [9]:
summary = [('items', describe(scripts, 'items')),
           ('quantity', describe(scripts, 'quantity')),
           ('nic', describe(scripts, 'nic')),
           ('act_cost', describe(scripts, 'act_cost'))]

In [ ]:
grader.score.pw__summary_statistics(summary)

## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [10]:
bnf_set=list()
for s in scripts:
    
    if s['bnf_name'] not in bnf_set:
        bnf_set.append(s['bnf_name'])
        
    

In [ ]:
len(bnf_set)

In [11]:

bnf_names = bnf_set
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [12]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)
    

In [ ]:
groups[bnf_names[0]]

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [13]:
max_item = [("", 0)]
temp={}
for key,values in groups.items():
    
    total_items=0
    for v in values:
        total_items=total_items+v['items']
    temp[key]=total_items
    

    

In [14]:
fin_max = max(temp, key=temp.get)

In [ ]:
temp[bnf_names[0]]

In [ ]:
fin_max

In [15]:
max_item = [(fin_max, temp[fin_max])]

In [ ]:
max_item

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [ ]:
grader.score.pw__most_common_item(max_item)

**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name',)` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [16]:

def group_by_field(data, fields):
    groups={}
    for field in fields:
        for script in scripts:
            if script[field] not in groups:
                groups[script[field]]=[script]
            else:
                groups[script[field]].append(script)
            
    
            
    return groups

In [ ]:
group_by_field(scripts, ('bnf_name',))

In [ ]:
groups = group_by_field(scripts, ('bnf_name',))
groups[bnf_names[0]]

In [17]:
groups = group_by_field(scripts, ('bnf_name',))
temp={}
for key,values in groups.items():
    
    total_items=0
    for v in values:
        total_items=total_items+v['items']
    temp[key]=total_items
    
fin_max = max(temp, key=temp.get)
test_max_item =  [(fin_max, temp[fin_max])]

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [18]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = min(practice_postal[practice['code']],practice['post_code'])
    else:
        practice_postal[practice['code']] = practice['post_code']

In [19]:
practice_postal["K82019"]

'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [20]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [21]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script["practice"]]

In [22]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [23]:
items_by_post = group_by_field(joined,("post_code",))

In [25]:
temp={}
for key,values in items_by_post.items():
    
    total_items=0
    for v in values:
        total_items=total_items+v['items']
    temp[key]=total_items

In [ ]:
temp

In [ ]:
len(temp)

In [26]:
results=[]
for k,v in temp.items():
    if v>0:
        results.append((k,v))

In [27]:
results.sort()

In [ ]:
postal_totals = [('B11 4BW', 20673)] * 100


grader.score.pw__postal_totals(results[:100])

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

There are several approaches to solve this problem but we will guide you through one of them. Feel free to solve it your own way if it is easier for you to understand and implement. If your kernel keeps on dying, it's probably an indication that you are running out of memory. Consider deleting objects you don't need anymore using the `del` statement and shutdown any other running notebooks. For example:
```Python
del some_object_not_needed
```

The first step is to calculate the total items for each `'post_code'` and `'bnf_name'`. Let's call that result `total_items_by_post_bnf`. Consider what is the best data structure(s) to represent `total_items_by_post_bnf`. It should have 141196 `('post_code', 'bnf_name')` groups.

In [28]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

In [29]:
group_by_post=group_by_field(joined,("post_code",))

In [30]:
type(group_by_post)

dict

In [38]:
from collections import defaultdict
final={}
for k,v in group_by_post.items():
    unique_bnf=defaultdict(int)
    for value in v:
        unique_bnf[value["bnf_name"]]=unique_bnf[value["bnf_name"]]+value["items"]
        
    final[k]=unique_bnf
    
        
            
    

In [40]:
final["SK11 6JL"]

defaultdict(int,
            {'Co-Magaldrox_Susp 195mg/220mg/5ml S/F': 5,
             'Alginate_Raft-Forming Oral Susp S/F': 3,
             'Sod Algin/Pot Bicarb_Susp S/F': 94,
             'Sod Alginate/Pot Bicarb_Tab Chble 500mg': 9,
             'Gaviscon Infant_Sach 2g (Dual Pack) S/F': 41,
             'Gaviscon Advance_Liq (Aniseed) (Reckitt)': 98,
             'Gaviscon Advance_Tab Chble Mint(Reckitt)': 16,
             'Gaviscon Advance_Liq (Peppermint) S/F': 65,
             'Peptac_Liq (Peppermint) S/F': 14,
             'Alverine Cit_Cap 60mg': 10,
             'Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp': 3,
             'Hyoscine Butylbrom_Tab 10mg': 153,
             'Mebeverine HCl_Tab 135mg': 84,
             'Mebeverine HCl_Cap 200mg M/R': 194,
             'Peppermint Oil_Cap E/C 0.2ml': 42,
             'Peppermint Oil_Cap E/C 0.2ml M/R': 5,
             'Colpermin_Cap E/C 0.2ml M/R': 6,
             'Ispag/Mebeverine_Gran Eff 3.5g/135mg S/F': 10,
             'Fybogel

In [42]:
max(final["SK11 6JL"], key=final["SK11 6JL"].get)

'Omeprazole_Cap E/C 20mg'

In [43]:
final["SK11 6JL"]['Omeprazole_Cap E/C 20mg']

3219

In [ ]:
for k,v in final.items():
    max(final[k],key=final.ge)

In [47]:
answer={}
for key,value in final.items():
    int_keys={}
    for k,v in value.items():
        if v in int_keys:
            int_keys[v]=min(int_keys[v],k)
        else:
            int_keys[v]=k
    
    answer[key]=int_keys
    
    

In [51]:
max(answer["SK11 6JL"])

3219

In [97]:
def sum_post(post_k):
    sum=0
    for li in group_by_post[post_k]:
        sum=sum+li["items"]
    return sum
    
    

In [64]:
plus=0
for j in joined:
    plus=plus+j["items"]

In [98]:
sum_post("B11 4BW")

20673

In [74]:
plus

4410054

In [99]:
submit=[]
for k,v in answer.items():
    submit.append((k,v[max(v)],max(v)/sum_post(k)))

    

In [100]:
submit[0]

('SK11 6JL', 'Omeprazole_Cap E/C 20mg', 0.029244760200234393)

In [101]:
submit.sort()

In [102]:
submit[0]

('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487)

In [62]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

In [59]:
grouped_by_post_bnf = group_by_field(joined,("post_code","bnf_name"))
temp={}
for key,values in grouped_by_post_bnf.items():
    
    total_items=0
    for v in values:
        total_items=total_items+v['items']
    temp[key]=total_items

In [ ]:
len(grouped_by_post_bnf)

In [ ]:


assert len(total_items_by_post_bnf) == 141196

Next, let's take `total_items_by_post_bnf` and group it by `'post_code'`. In other words, we want a  data structure that maps a `'post_code'` to a list of all records that belong to that `'post_code'`. There should be 118 groups.

In [ ]:
grouped_post_code = group_by_field(joined,("post_code",))

assert len(grouped_post_code) == 118

Now with `grouped_post_code`, let's iterate over each group and calculate the following fields for each `'post_code'`:
1. the sum of total items for all `'bnf_name'`
1. the most total items
1. the `'bnf_name'` that had the most total items

Once again, consider the best data structure(s) to use to represent the result. It may help to write and use a function when developing your solution.

In [ ]:
most_item_data_by_post = ...

Now, we are ready to:
1. calculate the ratio (the amount dispensed as proportion of total)
1. [sort](https://docs.python.org/3/howto/sorting.html) alphabetically by the post code
1. format the answer as a list of tuples
1. take only the first 100 tuples
1. submit to the grader

In [84]:
p=0
for li in group_by_post["B11 4BW"]:
    p=p+li["items"]

p

20673

In [85]:
submit[0]

('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0001600887426775273)

In [72]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [103]:
items_by_region=submit[:100]

In [104]:
grader.score.pw__items_by_region(items_by_region)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*